# Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import libraries

In [ ]:
import numpy as np
import pandas as pd
# Prevent AttributeError. via
# https://stackoverflow.com/a/76404841
pd.DataFrame.iteritems = pd.DataFrame.items
import statistics

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("dplyr")
install.packages("ggh4x")
devtools::install_github("hadley/ggplot2")
devtools::install_github("cran/GiniWegNeg")
devtools::install_github("kgwet/irrCAC")
install.packages("patchwork")
install.packages("showtext")
install.packages("stringr")
library(dplyr)
library(ggh4x)
library(ggplot2)
library(GiniWegNeg)
library(irrCAC)
library(patchwork)
library(showtext)
# Need to download Arial.ttf and upload it to your Google Drive prior.
font_add("Arial", regular = "/content/drive/MyDrive/Arial.ttf")
library(stringr)

# Import datasets

## Dataset with annotations and annotator information

In [ ]:
largeScale = pd.read_csv('/content/drive/MyDrive/datasets/largeScale.csv')
largeScale = largeScale.reset_index(drop=True)

## GPT annotations

In [ ]:
AWA_q1d_0301 = pd.read_csv(
    '/content/drive/MyDrive/datasets/cleaning/'
    'AWA_q1d_predictions_cleaned_gpt-3.5-turbo-0301.tsv',
    sep='\t')
AWA_q1d_0301 = AWA_q1d_0301.reset_index(drop=True)
AWA_q1d_0613 = pd.read_csv(
    '/content/drive/MyDrive/datasets/cleaning/'
    'AWA_q1d_predictions_cleaned_gpt-3.5-turbo-0613.tsv',
    sep='\t')
AWA_q1d_0613 = AWA_q1d_0613.reset_index(drop=True)

# Aggregate actual annotations by demographic group

In [ ]:
# Pandas Groupby and mode. via
# https://stackoverflow.com/a/54304691 and
# https://stackoverflow.com/a/10374456
AWA_q1d_age = largeScale.groupby(['tweet', 'annotatorAge'])['racist'].agg(
    lambda x: int(np.round(statistics.mean(x)))).to_frame(
    name='racist_mean').reset_index()
AWA_q1d_gender = largeScale.groupby(['tweet', 'annotatorGender'])['racist'].agg(
    lambda x: int(np.round(statistics.mean(x)))).to_frame(
    name='racist_mean').reset_index()
AWA_q1d_race = largeScale.groupby(['tweet', 'annotatorRace'])['racist'].agg(
    lambda x: int(np.round(statistics.mean(x)))).to_frame(
    name='racist_mean').reset_index()

# Define a function to compute and interpret Krippendorff's alpha coefficients between personas and actual annotators

In [ ]:
%%R
compute_alpha_ia <- function(df_gold, df_pred, demog_var, demog_spec, persona,
                             model, measure) {
    df_gold <- df_gold[df_gold[demog_var] == demog_spec, ]
    df <- merge(df_gold, df_pred[c("tweet", persona)], by = "tweet")
    df <- df[c("racist_mean", persona)]
    print(nrow(df))
    df <- as.matrix(sapply(df, as.numeric))
    # Retrieve and interpret Krippendorff's alpha values. via
    # https://cran.r-project.org/web/packages/irrCAC/vignettes/
    # benchmarking.html
    a <- krippen.alpha.raw(df,
        weights = "linear",
        categ.labels = c(1, 2, 3, 4, 5)
    )
    print(a$est)
    print(a$categories)
    print(landis.koch.bf(a$est$coeff.val, a$est$coeff.se))
    res <- data.frame(
        alpha = c(a$est$coeff.val),
        persona = c(persona),
        model = c(model),
        measure = c(measure)
    )
    return(res)
}

# Call the `compute_alpha_ia` function to compute and interpret Krippendorff's alpha coefficients between personas and actual annotators

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_18_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    18.0,
    "predicted_q1d_ratings_a_person_(aged_18-24)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_18_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_18_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    18.0,
    "predicted_q1d_ratings_a_person_(aged_18-24)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_18_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_25_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    25.0,
    "predicted_q1d_ratings_a_person_(aged_25-29)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_25_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_25_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    25.0,
    "predicted_q1d_ratings_a_person_(aged_25-29)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_25_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_30_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    30.0,
    "predicted_q1d_ratings_a_person_(aged_30-34)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_30_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_30_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    30.0,
    "predicted_q1d_ratings_a_person_(aged_30-34)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_30_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_35_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    35.0,
    "predicted_q1d_ratings_a_person_(aged_35-39)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_35_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_35_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    35.0,
    "predicted_q1d_ratings_a_person_(aged_35-39)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_35_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_40_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    40.0,
    "predicted_q1d_ratings_a_person_(aged_40-44)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_40_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_40_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    40.0,
    "predicted_q1d_ratings_a_person_(aged_40-44)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_40_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_45_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    45.0,
    "predicted_q1d_ratings_a_person_(aged_45-49)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_45_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_45_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    45.0,
    "predicted_q1d_ratings_a_person_(aged_45-49)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_45_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_50_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    50.0,
    "predicted_q1d_ratings_a_person_(aged_50-54)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_50_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_50_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    50.0,
    "predicted_q1d_ratings_a_person_(aged_50-54)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_50_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_55_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    55.0,
    "predicted_q1d_ratings_a_person_(aged_55-59)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_55_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_55_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    55.0,
    "predicted_q1d_ratings_a_person_(aged_55-59)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_55_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_60_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    60.0,
    "predicted_q1d_ratings_a_person_(aged_60-64)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_60_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_60_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    60.0,
    "predicted_q1d_ratings_a_person_(aged_60-64)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_60_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0301
alpha_ia_65_0301 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0301,
    "annotatorAge",
    65.0,
    "predicted_q1d_ratings_a_person_(aged_65+)_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_65_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_age -i AWA_q1d_0613
alpha_ia_65_0613 <- compute_alpha_ia(
    AWA_q1d_age,
    AWA_q1d_0613,
    "annotatorAge",
    65.0,
    "predicted_q1d_ratings_a_person_(aged_65+)_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_65_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_gender -i AWA_q1d_0301
alpha_ia_man_0301 <- compute_alpha_ia(
    AWA_q1d_gender,
    AWA_q1d_0301,
    "annotatorGender",
    "man",
    "predicted_q1d_ratings_a_man/male_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_man_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_gender -i AWA_q1d_0613
alpha_ia_man_0613 <- compute_alpha_ia(
    AWA_q1d_gender,
    AWA_q1d_0613,
    "annotatorGender",
    "man",
    "predicted_q1d_ratings_a_man/male_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_man_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_gender -i AWA_q1d_0301
alpha_ia_woman_0301 <- compute_alpha_ia(
    AWA_q1d_gender,
    AWA_q1d_0301,
    "annotatorGender",
    "woman",
    "predicted_q1d_ratings_a_woman/female_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_woman_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_gender -i AWA_q1d_0613
alpha_ia_woman_0613 <- compute_alpha_ia(
    AWA_q1d_gender,
    AWA_q1d_0613,
    "annotatorGender",
    "woman",
    "predicted_q1d_ratings_a_woman/female_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_woman_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_gender -i AWA_q1d_0301
alpha_ia_nonbinary_0301 <- compute_alpha_ia(
    AWA_q1d_gender,
    AWA_q1d_0301,
    "annotatorGender",
    "nonBinary",
    "predicted_q1d_ratings_a_non-binary_person_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_nonbinary_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_gender -i AWA_q1d_0613
alpha_ia_nonbinary_0613 <- compute_alpha_ia(
    AWA_q1d_gender,
    AWA_q1d_0613,
    "annotatorGender",
    "nonBinary",
    "predicted_q1d_ratings_a_non-binary_person_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_nonbinary_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_race -i AWA_q1d_0301
alpha_ia_white_0301 <- compute_alpha_ia(
    AWA_q1d_race,
    AWA_q1d_0301,
    "annotatorRace",
    "white",
    "predicted_q1d_ratings_a_White/Caucasian_person_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_white_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_race -i AWA_q1d_0613
alpha_ia_white_0613 <- compute_alpha_ia(
    AWA_q1d_race,
    AWA_q1d_0613,
    "annotatorRace",
    "white",
    "predicted_q1d_ratings_a_White/Caucasian_person_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_white_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_race -i AWA_q1d_0301
alpha_ia_black_0301 <- compute_alpha_ia(
    AWA_q1d_race,
    AWA_q1d_0301,
    "annotatorRace",
    "black",
    "predicted_q1d_ratings_a_Black/African_American_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_black_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_race -i AWA_q1d_0613
alpha_ia_black_0613 <- compute_alpha_ia(
    AWA_q1d_race,
    AWA_q1d_0613,
    "annotatorRace",
    "black",
    "predicted_q1d_ratings_a_Black/African_American_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_black_0613)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_race -i AWA_q1d_0301
alpha_ia_hisp_0301 <- compute_alpha_ia(
    AWA_q1d_race,
    AWA_q1d_0301,
    "annotatorRace",
    "hisp",
    "predicted_q1d_ratings_a_Hispanic/Latinx_person_in_the_United_States",
    "gpt-3.5-turbo-\"0301\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_hisp_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_race -i AWA_q1d_0613
alpha_ia_hisp_0613 <- compute_alpha_ia(
    AWA_q1d_race,
    AWA_q1d_0613,
    "annotatorRace",
    "hisp",
    "predicted_q1d_ratings_a_Hispanic/Latinx_person_in_the_United_States",
    "gpt-3.5-turbo-\"0613\"",
    "\"alpha\"[ia]"
)
print(alpha_ia_hisp_0613)

# Define a function to compute and interpret Krippendorff"s alpha coefficients between the personas and the no-persona defaults

In [ ]:
%%R
compute_alpha_ib <- function(df, reference, model, measure) {
    alpha_list <- c()
    for (i in seq_along(colnames(df))) {
        print(colnames(df)[i])
        # Retrieve and interpret Krippendorff"s alpha values. via
        # https://cran.r-project.org/web/packages/irrCAC/vignettes/
        # benchmarking.html
        df_temp <- df[, c(reference, colnames(df)[i])]
        df_temp <- as.matrix(sapply(df_temp, as.numeric))
        a <- krippen.alpha.raw(df_temp,
            weights = "linear",
            c(1, 2, 3, 4, 5)
        )
        print(a$est)
        print(a$categories)
        print(landis.koch.bf(a$est$coeff.val, a$est$coeff.se))
        if (i != 14) {
            alpha_list <- c(alpha_list, a$est$coeff.val)
        }
    }
    res <- data.frame(
        alpha = alpha_list,
        persona = colnames(df[, -14]),
        model = rep(model, length(alpha)),
        measure = rep(measure, length(alpha))
    )
    return(res)
}

# Call the `compute_alpha_ib` function to compute and interpret Krippendorff's alpha coefficients between the personas and the no-persona defaults

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_0301
alpha_ib_0301 <- compute_alpha_ib(AWA_q1d_0301[, 2:21], "predicted_q1d_ratings_base", "gpt-3.5-turbo-\"0301\"", "\"alpha\"[ib]")
print(alpha_ib_0301)

In [ ]:
# Import data between languages. via
# https://www.askpython.com/python/examples/
# use-r-and-python-in-the-same-notebook
%%R -i AWA_q1d_0613
alpha_ib_0613 <- compute_alpha_ib(AWA_q1d_0613[, 2:21], "predicted_q1d_ratings_base", "gpt-3.5-turbo-\"0613\"", "\"alpha\"[ib]")
print(alpha_ib_0613)

# Plot Krippendorff's alpha coefficients and Bias

In [ ]:
%%R
showtext_auto()
data <- rbind(
    alpha_ib_0301,
    alpha_ib_0613,
    alpha_ia_18_0301,
    alpha_ia_18_0613,
    alpha_ia_25_0301,
    alpha_ia_25_0613,
    alpha_ia_30_0301,
    alpha_ia_30_0613,
    alpha_ia_35_0301,
    alpha_ia_35_0613,
    alpha_ia_40_0301,
    alpha_ia_40_0613,
    alpha_ia_45_0301,
    alpha_ia_45_0613,
    alpha_ia_50_0301,
    alpha_ia_50_0613,
    alpha_ia_55_0301,
    alpha_ia_55_0613,
    alpha_ia_60_0301,
    alpha_ia_60_0613,
    alpha_ia_65_0301,
    alpha_ia_65_0613,
    alpha_ia_man_0301,
    alpha_ia_man_0613,
    alpha_ia_woman_0301,
    alpha_ia_woman_0613,
    alpha_ia_nonbinary_0301,
    alpha_ia_nonbinary_0613,
    alpha_ia_white_0301,
    alpha_ia_white_0613,
    alpha_ia_black_0301,
    alpha_ia_black_0613,
    alpha_ia_hisp_0301,
    alpha_ia_hisp_0613
)
# Replace spaces with underscores. via
# https://stackoverflow.com/a/53107084
data$persona <- gsub("_", " ", data$persona)
# Remove first few words. via
# https://stackoverflow.com/a/64889117
data$persona <- sub("^\\w+\\s\\w+\\s\\w+\\s\\w+\\s", "", data$persona)
# Remove everything after specified substring. via
# https://stackoverflow.com/a/53503384
data$persona <- sub("in the.+$", "", data$persona)
data$persona <- Map(paste, data$persona, "...")
# Compute bias
data$bias <- c(
    rep(Gini_RSV(alpha_ib_0301$alpha[1:10])[[1]], 10),
    rep(Gini_RSV(alpha_ib_0301$alpha[11:13])[[1]], 3),
    rep(Gini_RSV(alpha_ib_0301$alpha[14:19])[[1]], 6),
    rep(Gini_RSV(alpha_ib_0613$alpha[1:10])[[1]], 10),
    rep(Gini_RSV(alpha_ib_0613$alpha[11:13])[[1]], 3),
    rep(Gini_RSV(alpha_ib_0613$alpha[14:19])[[1]], 6),
    rep(0, 32)
)
data$labs <- c(
    rep(format(round(Gini_RSV(alpha_ib_0301$alpha[1:10])[[1]], digits = 3),
        nsmall = 3
    ), 10),
    rep(format(round(Gini_RSV(alpha_ib_0301$alpha[11:13])[[1]], digits = 3),
        nsmall = 3
    ), 3),
    rep(format(round(Gini_RSV(alpha_ib_0301$alpha[14:19])[[1]], digits = 3),
        nsmall = 3
    ), 6),
    rep(format(round(Gini_RSV(alpha_ib_0613$alpha[1:10])[[1]], digits = 3),
        nsmall = 3
    ), 10),
    rep(format(round(Gini_RSV(alpha_ib_0613$alpha[11:13])[[1]], digits = 3),
        nsmall = 3
    ), 3),
    rep(format(round(Gini_RSV(alpha_ib_0613$alpha[14:19])[[1]], digits = 3),
        nsmall = 3
    ), 6),
    rep("NA", 32)
)
data$attribute <- c(
    rep("Age~range", 10),
    rep("Gender~identity", 3),
    rep("Race/Ethnicity", 6),
    rep("Age~range", 10),
    rep("Gender~identity", 3),
    rep("Race/Ethnicity", 6),
    rep("Age~range", 20),
    rep("Gender~identity", 6),
    rep("Race/Ethnicity", 6)
)
data$grp <- c(rep(1, 70))
# Dot plots. via
# https://uc-r.github.io/cleveland-dot-plots
g1 <- ggplot(
    data,
    aes(alpha, grp)
) +
    geom_line(aes(group = grp),
        size = 0.05
    ) +
    geom_point(size = 0.1) +
    # Label max value dots. via
    # https://stackoverflow.com/a/48351513
    geom_text(
        data = . %>% group_by(measure, attribute, model) %>%
            filter(alpha == max(alpha)),
        # Wrap and truncate text. via
        # https://stackoverflow.com/a/73357493
        aes(label = str_wrap(persona, width = 22)),
        hjust = 0,
        family = "Arial",
        size = 7 / .pt,
        nudge_x = 0.01,
        lineheight = 1
    ) +
    scale_x_continuous(
        breaks = c(-.25, 0, .25, .5, .75, 1),
        limits = c(-.25, 1)
    ) +
    xlab("Krippendorff's alpha coefficient") +
    facet_nested(
        rows = vars(measure, attribute, model),
        switch = "y",
        labeller = label_parsed,
        strip = strip_nested(size = "variable")
    ) +
    theme(
        axis.text.y = element_blank(),
        axis.title.y = element_blank(),
        axis.ticks.y = element_blank(),
        axis.ticks = element_line(size = 0.1),
        axis.text = element_text(
            family = "Arial",
            size = 7
        ),
        strip.text.y.left = element_text(
            angle = 0,
            family = "Arial",
            size = 7
        ),
        strip.background = element_rect(
            colour = "white",
            fill = "#ECECEC"
        ),
        panel.grid = element_line(
            color = "black",
            size = 0.1,
            linetype = 2
        ),
        panel.grid.major.y = element_blank(),
        panel.grid.minor.y = element_blank(),
        panel.grid.minor.x = element_blank(),
        panel.spacing.y = unit(0, "lines"),
        panel.background = element_rect(
            color = "black",
            fill = "white",
            linewidth = 0.1
        ),
        text = element_text(
            family = "Arial",
            size = 7
        )
    )
data$alpha <- NULL
data$persona <- NULL
data <- data[!duplicated(data), ]
g2 <- ggplot(
    data,
    aes(grp, bias)
) +
    geom_text(aes(label = labs),
        hjust = -0.2,
        family = "Arial",
        size = 7 / .pt
    ) +
    coord_flip() +
    ylab("Bias") +
    scale_y_continuous(
        breaks = c(0.00, 1.00),
        limits = c(0, 1)
    ) +
    geom_bar(
        stat = "identity",
        fill = "black"
    ) +
    facet_nested(
        rows = vars(measure, attribute, model),
        labeller = label_parsed
    ) +
    theme(
        axis.text.y = element_blank(),
        axis.title.y = element_blank(),
        axis.ticks.y = element_blank(),
        axis.ticks = element_line(size = 0.1),
        axis.text = element_text(
            family = "Arial",
            size = 7
        ),
        strip.text = element_blank(),
        strip.background = element_blank(),
        panel.grid = element_line(
            color = "black",
            size = 0.1,
            linetype = 2
        ),
        panel.grid.major.y = element_blank(),
        panel.grid.minor.y = element_blank(),
        panel.grid.minor.x = element_blank(),
        panel.spacing.y = unit(0, "lines"),
        panel.background = element_rect(
            color = "black",
            fill = "white",
            linewidth = 0.1
        ),
        text = element_text(
            family = "Arial",
            size = 7
        )
    )
g1 + g2 + plot_layout(widths = c(5, 1))
ggsave(
    "/content/drive/MyDrive/datasets/cleaning/AWA_q1d.eps",
    height = 12,
    width = 18,
    units = "cm"
)